In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-trace-misc

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
## You will need to run jupyter from that directory.
## source $TEMP_BIN/_dir-aliases.bash
source _dir-aliases.bash
alias | wc -l

8


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1210

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-trace-misc/test-1210


In [6]:
# Count aliases proper
alias | wc -l

9


In [7]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

2


In [8]:
##1 SHOWS HISTORY OF BASH COMMANDS
alias hist='history $LINES'
hist

 2029  source _dir-aliases.bash
 2030  alias | wc -l
 2031  echo $?
 2032  ## NOTE: For reproducability, the directory name needs to be fixed
 2033  ## In place of $$, use a psuedo random number (e,g., 1210)
 2034  ## *** All output from one run to the next needs to be the same ***
 2035  ## temp_dir=$TMP/test-$$
 2036  temp_dir=$TMP/test-1210
 2037  mkdir -p "$temp_dir"
 2038  # TODO: /bin/rm -rvf "$temp_dir"
 2039  cd "$temp_dir"
 2040  pwd
 2041  #ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
 2042  alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"
 2043  echo $?
 2044  # Count aliases proper
 2045  alias | wc -l
 2046  echo $?
 2047  # Count functions
 2048  typeset -f | egrep '^\w+' | wc -l
 2049  echo $?
 2050  ##1 SHOWS HISTORY OF BASH COMMANDS
 2051  alias hist='history $LINES'
 2052  hist


In [9]:
##2 SHOWS HISTORY WITHOUT TIMESTAMPS
# Removes timestamp from history (e.g., " 1972  [2014-05-02 14:34:12] dir *py" => " 1972  dir *py")
# TEST: function hist { h | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }
# note: funciton used to simplify specification of quotes

# quiet-unalias h #COMMAND NOT FOUND
function h { hist | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }

h

 2037  mkdir -p "$temp_dir"
 2038  # TODO: /bin/rm -rvf "$temp_dir"
 2039  cd "$temp_dir"
 2040  pwd
 2041  #ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
 2042  alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"
 2043  echo $?
 2044  # Count aliases proper
 2045  alias | wc -l
 2046  echo $?
 2047  # Count functions
 2048  typeset -f | egrep '^\w+' | wc -l
 2049  echo $?
 2050  ##1 SHOWS HISTORY OF BASH COMMANDS
 2051  alias hist='history $LINES'
 2052  hist
 2053  echo $?
 2054  ##2 SHOWS HISTORY WITHOUT TIMESTAMPS
 2055  # Removes timestamp from history (e.g., " 1972  [2014-05-02 14:34:12] dir *py" => " 1972  dir *py")
 2056  # TEST: function hist { h | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }
 2057  # note: funciton used to simplify specification of quotes
 2058  # quiet-unalias h #COMMAND NOT FOUND
 2059  function h { hist | perl -pe 's/^(\s*\d+\s*)(\[[^\]]+\])(.*)/$1$3/;'; }
 2060  h


In [10]:
##3 ASCTIME
# EX: $ asctime | perl -pe 's/\d/N/g; s/\w+ \w+/DDD MMM/;' => "DDD MMM NN NN:NN:NN NNNN"
function asctime() { perl -e "print (scalar localtime($1));"; echo ""; }

asctime | perl -pe 's/\d/N/g; s/\w+ \w+/DDD MMM/;'

DDD MMM NN NN:NN:NN NNNN


In [18]:
##4 FILTER DIRECTORY NAMES
# filter-dirnames: strip directory names from ps listing (TODO: rename as strip-dirnames)
function filter-dirnames () { perl -pe 's/\/[^ \"]+\/([^ \/\"]+)/$1/g;'; }

ps | filter-dirnames

    PID TTY          TIME CMD
   5235 pts/2    00:00:00 bash
   5375 pts/2    00:00:00 ps
   5376 pts/2    00:00:00 bash
   5377 pts/2    00:00:00 perl


In [19]:
##5 COMMAIZE NUMBERS
# comma-ize-number(): add commas to numbers in stdin
# EX: echo "1234567890" | comma-ize-number => 1,234,567,890
function comma-ize-number () { perl -pe 'while (/\d\d\d\d/) { s/(\d)(\d\d\d)([^\d])/\1,\2\3/g; } '; }

echo "99012342305324254" | comma-ize-number

99,012,342,305,324,254


In [20]:
##6 APPLYING NUMERIC SUFFIXES

# apply-numeric-suffixes([once=0]): converts numbers in stdin to use K/M/G suffixes.
# Notes:
# - K, M, G and T based on powers of 1024.
# - If $once non-zero, then the substitution is only applied one-time per line.
# - The number must be preceded by a word boundary and followed by whitespace.
# This was added in support of the usage function (e.g., numeric subdirectory names).
# TODO:
# - Convert to Perl script to avoid awkward bash command line construction.
# - Make the trailing context a word-boundry as well (rather than whitespace).
# EX: echo "1024 1572864 1073741824" | apply-numeric-suffixes => 1K 1.5M 1G
# EX: echo "1024 1572864 1073741824" | apply-numeric-suffixes 1 => 1K 1572864 1073741824
function apply-numeric-suffixes () {
    local just_once="$1"
    local g="g";
    if [ "$just_once" = "1" ]; then g=""; fi
    # TODO: make only sure the first number is converted if just-once applies
    # NOTE: 3 args to sprintf: coefficient, KMGT suffix, and post-context
    ## DEBUG; perl -pe '$suffixes="_KMGT";  s@\b(\d{4,15})(\s)@$pow = log($1)/log(1024);  $new_num=($1/1024**$pow);  $suffix=substr($suffixes, $pow, 1);  print STDERR ("s=$suffixes p=$pow nn=$new_num l=$suffix\n"); sprintf("%.3g%s%s", $new_num, $suffix, $2)@e'"$g;"
    perl -pe '$suffixes="_KMGT";  s@\b(\d{4,15})(\s)@$pow = int(log($1)/log(1024));  $new_num=($1/1024**$pow);  $suffix=substr($suffixes, $pow, 1);  sprintf("%.3g%s%s", $new_num, $suffix, $2)@e'"$g;"
}

# TESTED COMMAND (if 1 KB ~ 1000 B; must return 7.XX GB (~ 8 GB) )
echo "8000000000" | apply-numeric-suffixes

7.45G


In [21]:
##6.1 APPLYING USAGE NUMERIC SUFFIXES
# apply-usage-numeric-suffixes(): factors in 1k blocksize before applying numeric suffixes
# note: (?=\s) is lookahead pattern (see perlre manpage)
#
function apply-usage-numeric-suffixes() {
    perl -pe 's@^(\d+)(?=\s)@$1 * 1024@e;' | apply-numeric-suffixes 1
}

# IN SHORT = result * 1000
echo "8000000000" | apply-usage-numeric-suffixes #G -> T
echo "8000000" | apply-usage-numeric-suffixes #M -> G

7.45T
7.63G


In [22]:
#7 usage-pp
function usage {
    ## TODO: output_file=(("$1"||"usage.list"));
    output_file=$(default_assignment "$1" "usage.list")
    rename-with-file-date "$output_file";
    $NICE du --block-size=1K --one-file-system 2>&1 | $NICE sort -rn | apply-usage-numeric-suffixes >| $output_file 2>&1;
    $PAGER $output_file;
}
function usage-alt {
    local output_file=$TEMP/$(basename $PWD)"-usage.list";
    usage "$output_file"
}

function byte-usage () { output_file="usage.bytes.list"; backup-file $output_file; $NICE du --bytes --one-file-system 2>&1 | $NICE sort -rn | apply-usage-numeric-suffixes >| $output_file 2>&1; $PAGER $output_file; }
## TODO: function usage () { du --one-file-system --human-readable 2>&1 | sort -rn >| usage.list 2>&1; $PAGER usage.list; }

#TODO: rework so that pp version saved in file
alias usage-pp='usage | apply-usage-numeric-suffixes | $PAGER'
#

In [23]:
# ERROR GENERATED- MULTIPLE
# $ ups a > psa.txt
# $ usage psa.txt
# | bash: ups: command not found
# | bash: default_assignment: command not found
# | bash: rename-with-file-date: command not found
# | bash: $output_file: ambiguous redirect
# | sort: fflush failed: 'standard output': Broken pipe
# | sort: write error

In [24]:
#8 number-columns(file): number each column in first line of tabular file
function number-columns () { head -1 "$1" | perl -0777 -pe '$c = 1; s/^/1: /; s/\t/"\t" . ++$c . ": "/eg;'; }
# TODO: s/\t/\\t/g;
function number-columns-comma () { head -1 "$1" | perl -pe 's/,/\t/g;' | number-columns -; }


In [25]:
#8.1) 
printf "THIS IS THE START\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS A TEST\nTHIS IS THE END\n" > thisisatest.txt
ps aux > process.txt

number-columns thisisatest.txt
number-columns-comma process.txt

1: THIS IS THE START
1: USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND


In [26]:
#9) reverse REVERSES cat COMMAND
# alias type='cat'  # interferes with type command
alias reverse='tac'
cat thisisatest.txt
linebr
reverse thisisatest.txt

THIS IS THE START
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS THE END
--------------------------------------------------------------------------------
THIS IS THE END
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS A TEST
THIS IS THE START


In [27]:
#10) backup-file
function backup-file () { local file="$1"; if [ -e "$file" ]; then dobackup.sh "$file"; fi; }
## TODO: output header (e.g., "num-blocks<TAB>dir    # note: blocksize is 1k")
#

In [29]:
# ERROR - NEED A CHANGE OF PATH (TBF) ==solved==
backup-file thisisatest.txt
# bash: /home/aveey/bin/dobackup.sh: /bin/csh: bad interpreter: No such file or directory

Backing up 'thisisatest.txt' to './backup/thisisatest.txt'


In [30]:
function byte-usage () { output_file="usage.bytes.list"; backup-file $output_file; $NICE du --bytes --one-file-system 2>&1 | $NICE sort -rn | apply-usage-numeric-suffixes >| $output_file 2>&1; $PAGER $output_file; }
## TODO: function usage () { du --one-file-system --human-readable 2>&1 | sort -rn >| usage.list 2>&1; $PAGER usage.list; }


In [34]:
# ERROR - NEED CHANGE OF PATH (TBF) ==solved==

byte-usage process.txt
# | bash: /home/aveey/bin/dobackup.sh: /bin/csh: bad interpreter: No such file or directory
# | 30.4M	.


# (Same output for 'byte-usage "Hi Mom"' and 'byte-usage thisisatest.txt')


Backing up 'usage.bytes.list' to './backup/usage.bytes.list'
35.9M	.
4.13M	./backup


In [35]:
# check[all-]-(errors|warnings): check for known errors, with the check-all
# variant including more patterns and with warnings subsuming errors.
## OLD:
## function check-errors () { (check_errors.perl -context=5 "$@") 2>&1 | $PAGER; }
## function check-all-errors () { (check_errors.perl -warnings -relaxed -context=5 "$@") 2>&1 | $PAGER; }
function check-errors () { (DEBUG_LEVEL=1 check_errors.perl -context=5 "$@") 2>&1 | $PAGER; }
function check-all-errors () { (DEBUG_LEVEL=1 check_errors.perl -warnings -relaxed -context=5 "$@") 2>&1 | $PAGER; }
alias check-warnings='echo "*** Error: use check-all-errors instead ***"; echo "    check-all-errors"'
alias check-all-warnings='check-all-errors -strict'
#
# check-errors-excerpt(log-file): show errors are start of log-file and at end if different
function check-errors-excerpt () {
    local base="$TMP/check-errors-excerpt-$$"
    local head="$base.head"
    local tail="$base.tail"
    check-errors "$@" | head >| $head;
    cat "$head"
    check-errors "$@" | tail >| $tail;
    diff "$head" "$tail" >| /dev/null
    if [ $? != 0 ]; then
        echo "\$?=$?"
        cat "$tail";
    fi
}

In [36]:
#11 - LOG FILES ARE REQUIRED FOR check-errors
check-errors process.txt
check-all-errors process.txt
linebr
check-warnings
linebr
check-all-warnings
linebr
check-errors-excerpt process.txt

process.txt
process.txt
--------------------------------------------------------------------------------
*** Error: use check-all-errors instead ***
    check-all-errors
--------------------------------------------------------------------------------

usage: check_errors.perl [options]

options = [-warnings] [-context=N] [-no_astericks] [-skip_ruby_lib] [-relaxed | -strict]

ex: check_errors.perl whatever


Notes:
- The default context is 1
- Warnings are skipped by default
- Use -no_astericks if input uses ***'s outside of error contexts
Use -relaxed to exclude special cases (e.g., xyz='error')

--------------------------------------------------------------------------------
process.txt


In [37]:
alias rdiff='rev_vdiff.sh'
alias tkdiff-='tkdiff -noopt'
function kdiff () { kdiff.sh "$@" & }
alias vdiff='kdiff'

In [41]:
# # 12) rdiff, tkdiff, vdiff - ERROR IN COMMANDS (as shown)

# $ rdiff process.txt psa.txt
# $ linebr
# $ tkdiff- process.txt psa.txt
# $ linebr
# $ vdiff process.txt 

# | Echo: Command not found.
# | ???
# | cvs: Command not found.
# | 
# | echo: No match.
# | echo: No match.
# | issuing: tkdiff.tcl -r.-1 process.txt
# | [1] 5609
# | tkdiff.tcl: Command not found.
# | --------------------------------------------------------------------------------
# | bash: tkdiff: command not found
# | --------------------------------------------------------------------------------
# | [1] 5615
